# Clasificar el nivel de gasto del cliente

## Objetivo

El objetivo del modelo es clasificar a cada cliente en dos categorías según su nivel de gasto:

Bajo: importe < 5.000

Alto: importe ≥ 5.000

Este es un problema de clasificación binaria, ya que se busca asignar cada registro a una de dos clases posibles.

## Algoritmo elegido: Regresion logística

Se selecciona el algoritmo de Regresión Logística porque está diseñado específicamente para problemas de clasificación binaria.

## Variables

### variable de salida (y) 


La variable objetivo será una variable binaria creada a partir del importe:

y = 1 → Cliente de gasto alto (importe ≥ 5000)

y = 0 → Cliente de gasto bajo (importe < 5000)

### Variables de entrada (X)

Están compuestas por todas las características independientes que pueden influir en la predicción de la clase.

X: 

## Métricas de evaluación

### Accuracy (Exactitud)

Porcentaje de predicciones correctas sobre el total.

### Matriz de confusión

Permite ver cuántos casos fueron correctamente y incorrectamente clasificados.

#### Librerías

In [1]:
%%capture
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%pip install scikit-learn
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, classification_report


In [2]:
datos = pd.read_csv('datasets/dataset_unificado/ventas_completo.csv')

In [3]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_venta         339 non-null    int64  
 1   fecha            339 non-null    object 
 2   id_cliente       339 non-null    int64  
 3   medio_pago       339 non-null    object 
 4   nombre_cliente   339 non-null    object 
 5   email            339 non-null    object 
 6   ciudad           339 non-null    object 
 7   fecha_alta       339 non-null    object 
 8   id_producto      339 non-null    int64  
 9   cantidad         339 non-null    int64  
 10  importe          339 non-null    float64
 11  nombre_producto  339 non-null    object 
 12  categoria        339 non-null    object 
 13  precio_unitario  339 non-null    float64
dtypes: float64(2), int64(4), object(8)
memory usage: 37.2+ KB


Importe es la variable objetivo

In [ ]:
datos['importe'] = np.where(datos.importe < 5000, 0, 1)
# Una vez creada la nueva variable respuesta se descarta la original
datos = datos.drop(columns = 'importe')